In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5, ))
])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 49.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.56MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 13.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.19MB/s]


In [8]:
class SimpleDNN(nn.Module):
  def __init__(self):
    super(SimpleDNN, self).__init__()
    self.fc1 = nn.Linear(28*28, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)
    self.relu = nn.ReLU()
    self.Dropout = nn.Dropout(0.2)
  def forward(self, x):
    x = x.view(-1, 28*28)
    x = self.relu(self.fc1(x))
    x = self.Dropout(x)
    x = self.relu(self.fc2(x))
    x = self.Dropout(x)
    x = self.fc3(x)
    return x

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleDNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(device)

cpu


In [11]:
epochs = 5
for epoch in range(epochs):
  running_loss = 0.0
  for images , labels in trainloader:
    inputs, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}')

Epoch 1/5, Loss: 0.3979
Epoch 2/5, Loss: 0.2053
Epoch 3/5, Loss: 0.1681
Epoch 4/5, Loss: 0.1425
Epoch 5/5, Loss: 0.1281


In [15]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
  for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 96.68%
